In [41]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import re
from tqdm import tqdm
from pathlib import Path
from itertools import islice

import numpy as np

import langdetect
import tokenize_uk
from gensim.models import KeyedVectors

from utils import lemmatize
from repository import Document, Repository1551

In [2]:
uk_vectors_file = '../data/vectors/news.lowercased.tokenized.word2vec.300d'
uk_vectors = KeyedVectors.load_word2vec_format(uk_vectors_file, binary=False)

In [3]:
def read_data_file(fpath):
    label = fpath.stem
    docs = list()
    with fpath.open('r') as f:
        data = f.read()
        data = re.split(r'\n\n\n', data)
        for d in data:
            try:
                id_, text = d.split('\n', 1)
                id_ = int(id_)
            except ValueError:
                pass
            else:
                if langdetect.detect(text[:200]) == 'uk':
                    lem = lemmatize(text)
                    doc = Document(id_=int(id_), text=text, text_lem=lem, label=label)
                    docs.append(doc)
    return docs

In [68]:
data_path = Path('../data/1551')

n_files = sum(1 for _ in data_path.iterdir())

part_files = islice(data_path.iterdir(), 0, 3)

r1551 = Repository1551()
for data_file in tqdm(data_path.iterdir(), total=n_files):
    docs = read_data_file(data_file)
    r1551.extend(*docs)

r1551.shuffle()

100%|██████████| 177/177 [18:10<00:00,  5.94s/it]


In [69]:
# щоб використати в наступнії домашці
r1551.save('../data/r1551.pickle')

In [70]:
from collections import Counter
Counter(r1551.labels)

Counter({'Недостатній-тиск-ГВП': 131,
         'Знищення-омели--амброзії-та-рослин---паразитів': 284,
         'Відсутність-опалення': 3137,
         'Контроль-за-станом-рекламних-засобів': 315,
         'Демонтаж-рекламних-вивісок-з-опорних-стовпів': 114,
         'Прибирання-дерев--гілок--листя-з-закріпленої-території': 181,
         'Укладання-та-ремонт-асфальтного-покриття': 3615,
         'Відсутність-опалення-по-стояку': 981,
         'Вологе-прибирання-приміщень': 405,
         'Незадовільна-температура-опалення': 624,
         'Перевірка-дозвільної-документації--демонтаж-кіосків--ларків': 2186,
         'Будівництво-дооблаштування-дитячого-майданчику': 630,
         'Скошування-трави': 72,
         'Розміщення-паркувальних-майданчиків': 150,
         'Встановлення-сміттєвих-контейнерів-та-урн-для-сміття': 346,
         'Відсутність-ГВП': 6546,
         'Відсутнє-ХВП': 780,
         'Встановлення-огородження-зеленої-зони': 285,
         'Незадовільна-температура-ГВП': 1109,
    

### Побудова бейзлайну
класифікатор kNN, який буде шукати найближчі за косинусною відстанню вектори запитів, утворені сумуванням векторів всіх слів запиту

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [72]:
def build_vector_matrics(r1551, use_lemma=False):
    x_data = np.zeros((len(r1551.texts), 300))
    
    if use_lemma:
        texts = r1551.lems
    else:
        texts = r1551.texts

    for i, text in tqdm(enumerate(texts), total=len(texts)):
        tokens = tokenize_uk.tokenize_words(text)
        for t in tokens:
            try:
                vec = uk_vectors.get_vector(t)
            except KeyError:
                pass
            else:
                x_data[i, :] += vec
        x_data[i, :] /= len(tokens)
    return x_data

In [73]:
X_CACHE = dict()

In [74]:
def pseudo_pipe(r1551, use_lemma, model):
    if f'x_lemma_{use_lemma}' in X_CACHE:
        x_data = X_CACHE[f'x_lemma_{use_lemma}']
    else:
        x_data = build_vector_matrics(r1551, False)
        X_CACHE[f'x_lemma_{use_lemma}'] = x_data

    x_train, x_test, y_train, y_test = train_test_split(x_data, r1551.labels, test_size=0.2, stratify=r1551.labels, random_state=42)
    
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    model.fit(x_train, y_train)
    predict = model.predict(x_test)

    report = classification_report(y_test, predict)
    print(report)

In [75]:
pseudo_pipe(r1551, use_lemma=False, model=KNeighborsClassifier(n_neighbors=5))

100%|██████████| 58207/58207 [00:26<00:00, 2165.70it/s]


                                                                             precision    recall  f1-score   support

                                                                Інші-Подяки       0.16      0.13      0.14        61
                                         Інші-технічні-недоліки-стану-ліфту       0.16      0.52      0.24       139
           Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.04      0.09      0.06        56
                                     Бажаючі-отримати--Картки-киянина--КК--       0.38      0.36      0.37        14
                         Будівництво--дооблаштування-спортивних-майданчиків       0.09      0.19      0.12        16
                                                            Будівництво-АЗС       0.04      0.06      0.05        18
                                                   Будівництво-в-нічний-час       0.09      0.20      0.12        10
                             Будівництво-дооблаштування-дитячог

In [76]:
pseudo_pipe(r1551, use_lemma=True, model=KNeighborsClassifier(n_neighbors=10))

100%|██████████| 58207/58207 [00:35<00:00, 1628.88it/s]


                                                                             precision    recall  f1-score   support

                                                                Інші-Подяки       0.24      0.13      0.17        61
                                         Інші-технічні-недоліки-стану-ліфту       0.18      0.53      0.27       139
           Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.05      0.05      0.05        56
                                     Бажаючі-отримати--Картки-киянина--КК--       0.43      0.21      0.29        14
                         Будівництво--дооблаштування-спортивних-майданчиків       0.25      0.25      0.25        16
                                                            Будівництво-АЗС       0.03      0.06      0.04        18
                                                   Будівництво-в-нічний-час       0.00      0.00      0.00        10
                             Будівництво-дооблаштування-дитячог

### Інші моделі

In [77]:

pseudo_pipe(r1551, use_lemma=False, model=LogisticRegression(C=1, class_weight='balanced'))

pseudo_pipe(r1551, use_lemma=True, model=LogisticRegression(C=1, class_weight='balanced'))


                                                                             precision    recall  f1-score   support

                                                                Інші-Подяки       0.33      0.30      0.31        61
                                         Інші-технічні-недоліки-стану-ліфту       0.58      0.41      0.48       139
           Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.17      0.12      0.14        56
                                     Бажаючі-отримати--Картки-киянина--КК--       0.45      0.71      0.56        14
                         Будівництво--дооблаштування-спортивних-майданчиків       0.30      0.50      0.37        16
                                                            Будівництво-АЗС       0.34      0.72      0.46        18
                                                   Будівництво-в-нічний-час       0.17      0.30      0.21        10
                             Будівництво-дооблаштування-дитячог

In [78]:

pseudo_pipe(r1551, use_lemma=False, model=RandomForestClassifier(n_estimators=100, class_weight='balanced'))

pseudo_pipe(r1551, use_lemma=True, model=RandomForestClassifier(n_estimators=100, class_weight='balanced'))


                                                                             precision    recall  f1-score   support

                                                                Інші-Подяки       0.70      0.11      0.20        61
                                         Інші-технічні-недоліки-стану-ліфту       0.51      0.29      0.37       139
           Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.70      0.12      0.21        56
                                     Бажаючі-отримати--Картки-киянина--КК--       0.83      0.36      0.50        14
                         Будівництво--дооблаштування-спортивних-майданчиків       1.00      0.12      0.22        16
                                                            Будівництво-АЗС       1.00      0.06      0.11        18
                                                   Будівництво-в-нічний-час       0.00      0.00      0.00        10
                             Будівництво-дооблаштування-дитячог

### Doc2Vec

In [79]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [80]:
x_train, x_test, y_train, y_test = train_test_split(r1551.lems, r1551.labels, test_size=0.2, stratify=r1551.labels, random_state=42)
documents = [TaggedDocument(t.split(' '), [i]) for i, t in enumerate(x_train)]

doc_model = Doc2Vec(vector_size=300, min_count=2, epochs=40)
doc_model.build_vocab(documents)
doc_model.train(documents, total_examples=doc_model.corpus_count, epochs=doc_model.epochs)

In [81]:
x_train = np.array([doc_model.infer_vector(t.split(' ')) for t in x_train])
x_test = np.array([doc_model.infer_vector(t.split(' ')) for t in x_test])

In [82]:
def pseudo_pipe(data, model):
    x_train, x_test, y_train, y_test = data
    
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    model.fit(x_train, y_train)
    predict = model.predict(x_test)

    report = classification_report(y_test, predict)
    print(report)

In [83]:
pseudo_pipe((x_train, x_test, y_train, y_test), KNeighborsClassifier(n_neighbors=5))

                                                                             precision    recall  f1-score   support

                                                                Інші-Подяки       0.06      0.25      0.10        61
                                         Інші-технічні-недоліки-стану-ліфту       0.20      0.31      0.24       139
           Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.07      0.20      0.10        56
                                     Бажаючі-отримати--Картки-киянина--КК--       0.17      0.21      0.19        14
                         Будівництво--дооблаштування-спортивних-майданчиків       0.00      0.00      0.00        16
                                                            Будівництво-АЗС       0.08      0.06      0.06        18
                                                   Будівництво-в-нічний-час       0.33      0.10      0.15        10
                             Будівництво-дооблаштування-дитячог

In [84]:
pseudo_pipe((x_train, x_test, y_train, y_test), LogisticRegression(class_weight='balanced'))

                                                                             precision    recall  f1-score   support

                                                                Інші-Подяки       0.10      0.23      0.14        61
                                         Інші-технічні-недоліки-стану-ліфту       0.51      0.21      0.30       139
           Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.13      0.07      0.09        56
                                     Бажаючі-отримати--Картки-киянина--КК--       0.10      0.57      0.17        14
                         Будівництво--дооблаштування-спортивних-майданчиків       0.11      0.19      0.14        16
                                                            Будівництво-АЗС       0.19      0.56      0.29        18
                                                   Будівництво-в-нічний-час       0.11      0.40      0.17        10
                             Будівництво-дооблаштування-дитячог